# Importing Library

In [1]:
import pandas as pd
import numpy as np
import requests
import folium

# Data Sourcing

## Bike Traffic Count Data Loading and Preprocessing

In [3]:
#Read csv and concatinate
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
for year in years:
    df = pd.concat([pd.read_csv(f"data/bike_counter/Jahresdatei {year}.csv", sep=';') for i in range(len(years))])

#Reset index    
df = df.reset_index(drop=True)

#Fill 0 in Nan 
df = df.fillna(0)

#Convert the counts from float to int
df[list(df.keys()[1:])] = df[list(df.keys()[1:])].astype(int)

In [4]:
df.keys()

Index(['Zählstelle        Inbetriebnahme', '02-MI-JAN-N 01.04.2015',
       '02-MI-JAN-S 01.04.2015', '03-MI-SAN-O 01.06.2015',
       '03-MI-SAN-W 01.06.2015', '05-FK-OBB-O 01.06.2015',
       '05-FK-OBB-W 01.06.2015', '06-FK-FRA-O 01.06.2016',
       '06-FK-FRA-W 01.06.2016', '10-PA-BER-N 01.05.2016',
       '10-PA-BER-S 01.05.2016', '12-PA-SCH 01.03.2012',
       '13-CW-PRI 01.04.2015', '15-SP-KLO-N 01.06.2016',
       '15-SP-KLO-S 01.06.2016', '17-SZ-BRE-O 01.05.2016',
       '17-SZ-BRE-W 01.05.2016', '18-TS-YOR-O 01.04.2015',
       '18-TS-YOR-W 01.04.2015', '19-TS-MON 01.05.2015',
       '20-TS-MAR-N  01.05.2016', '20-TS-MAR-S 01.05.2016',
       '21-NK-MAY 01.05.2016', '23-TK-KAI 01.05.2016', '24-MH-ALB 01.07.2015',
       '26-LI-PUP 01.06.2015', '27-RE-MAR 01.05.2015', '06-CW-JU-O 27.07.2021',
       '06-CW-JU-W 27.07.2021', '02-MI-AL-W 16.12.2021'],
      dtype='object')

In [5]:
#Renaming columns
d_rename = {
    'Zählstelle        Inbetriebnahme': 'datetime',
    '02-MI-JAN-N 01.04.2015': '02-N',
    '02-MI-JAN-S 01.04.2015': '02-S', 
    '03-MI-SAN-O 01.06.2015': '03-O',
    '03-MI-SAN-W 01.06.2015': '03-W',
    '05-FK-OBB-O 01.06.2015': '05-O',
    '05-FK-OBB-W 01.06.2015': '05-W',
    '06-FK-FRA-O 01.06.2016': '06-O',
    '06-FK-FRA-W 01.06.2016': '06-W',
    '10-PA-BER-N 01.05.2016': '10-N',
    '10-PA-BER-S 01.05.2016': '10-S', 
    '12-PA-SCH 01.03.2012': '12',
    '13-CW-PRI 01.04.2015': '13', 
    '15-SP-KLO-N 01.06.2016': '15-N',
    '15-SP-KLO-S 01.06.2016': '15-S', 
    '17-SZ-BRE-O 01.05.2016': '17-O',
    '17-SZ-BRE-W 01.05.2016': '17-W', 
    '18-TS-YOR-O 01.04.2015': '18-O',
    '18-TS-YOR-W 01.04.2015': '18-W', 
    '19-TS-MON 01.05.2015': '19',
    '20-TS-MAR-N  01.05.2016': '20-N',
    '20-TS-MAR-S 01.05.2016': '20-S',
    '21-NK-MAY 01.05.2016': '21',
    '23-TK-KAI 01.05.2016': '23',
    '24-MH-ALB 01.07.2015': '24',
    '26-LI-PUP 01.06.2015': '26', 
    '27-RE-MAR 01.05.2015': '27'
    }
df = df.rename(columns=d_rename)

In [7]:
df

,datetime,02-N,02-S,03-O,03-W,05-O,05-W,06-O,06-W,10-N,...,20-N,20-S,21,23,24,26,27,06-CW-JU-O 27.07.2021,06-CW-JU-W 27.07.2021,02-MI-AL-W 16.12.2021
0,"Fri, 1/ Jan 2021 00:00",5,6,4,2,0,0,8,5,5,...,0,2,29,6,4,15,6,0,0,0
1,"Fri, 1/ Jan 2021 01:00",21,14,12,18,0,0,24,7,23,...,2,0,51,7,13,21,4,0,0,0
2,"Fri, 1/ Jan 2021 02:00",28,37,13,17,0,0,26,23,24,...,1,1,51,6,5,25,11,0,0,0
3,"Fri, 1/ Jan 2021 03:00",36,27,10,14,0,0,24,18,20,...,1,3,41,5,3,18,9,0,0,0
4,"Fri, 1/ Jan 2021 04:00",24,24,5,8,0,0,12,14,19,...,1,3,30,6,4,15,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87595,"Fri, 31/ Dec 2021 19:00",32,27,21,29,0,0,3,29,19,...,0,4,74,4,10,17,14,13,18,22
87596,"Fri, 31/ Dec 2021 20:00",17,21,14,14,0,0,2,10,13,...,0,3,56,15,1,13,13,7,8,8
87597,"Fri, 31/ Dec 2021 21:00",13,23,15,15,0,0,2,8,14,...,0,1,26,7,4,4,6,2,8,20
87598,"Fri, 31/ Dec 2021 22:00",25,15,8,11,0,0,2,13,7,...,0,1,19,6,2,5,9,5,9,9
